In [5]:
import os
import cv2
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import scipy.io as sio
import plotly.io as pio
import plotly.express as px
import pandas as pd

from my_funcs.plot_functions import mask_roi_finder
from my_funcs.plot_functions import mask_check_plot
from my_funcs.plot_functions import compare_txt_method

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.cest_functions import z_spec_rearranger
from my_funcs.cest_functions import offset_rearranger

from my_funcs.b0_mapping_functions import wassr_b0_mapping
from my_funcs.b0_mapping_functions import b0_correction


## 1.1. Enter Data: ##

In [6]:
phantom_choice = 1

# Root stats:
general_fn = os.path.abspath(os.curdir)
# Subject stats:
glu_phantom_fns = [os.path.join(general_fn, 'scans', 'glu_phantoms_temp_var_feb12_2024',
                                '20240212_091535_or_perlman_glu_phantom_20_16_12mM_ph_7_dec_33deg'),
                   os.path.join(general_fn, 'scans', 'glu_phantoms_temp_var_feb12_2024',
                                '20240212_103350_or_perlman_glu_phantom_20_16_12mM_ph_7_dec_37deg'),
                   os.path.join(general_fn, 'scans', 'glu_phantoms_temp_var_feb12_2024',
                                '20240212_114515_or_perlman_glu_phantom_20_16_12mM_ph_7_dec_40deg'),
                   os.path.join(general_fn, 'scans', 'glu_phantoms_temp_var_feb12_2024',
                                '20240212_132706_or_perlman_glu_phantom_20_16_12mM_ph_7_feb_33deg'),
                   os.path.join(general_fn, 'scans', 'glu_phantoms_temp_var_feb12_2024',
                                '20240212_132706_or_perlman_glu_phantom_20_16_12mM_ph_7_feb_37deg'),
                   os.path.join(general_fn, 'scans', 'hagar_glutamate_phantoms37deg_dec8_2023',
                                '20231207_093147_Or_Perlman_phantom_glu_ph7_12_16_20mM_37deg_1_1')]  # February
glu_phantom_fn = glu_phantom_fns[phantom_choice-1]
txt_file_name = 'labarchive_notes.txt'


In [7]:
# _, files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, 'EPI_4uT')
# _, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
# e_r = 0
# vial_rois, full_mask, _ = mask_roi_finder(bruker_dataset_mask, e_r)
#     
# dict_fn = os.path.join(files_fn, 'dict.mat')
# quant_fn = os.path.join(files_fn, 'quant_maps.mat')
# zspec_dict = sio.loadmat(dict_fn)
# zspec_quant = sio.loadmat(quant_fn)
# 
# t1w = zspec_dict['t1w'][0]
# t2w = zspec_dict['t2w'][0]
# fs = zspec_dict['fs_0'][0]
# ksw = zspec_dict['ksw_0'][0]
# sig = zspec_dict['sig']
# m_id = zspec_quant['match_id']
# vial_20_m = m_id[46,31] # phantom 1
# # vial_10_m = m_id[28,47] # phantom 1
# vial_10_m = m_id[27,19] # phantom 2
# 
# py_df = pd.DataFrame({
#         't1w': t1w,
#         't2w': t2w,
#         'fs': fs,
#         'ksw': ksw,
#     })
# py_df_sig = pd.DataFrame(sig)
# # Concatenate the existing DataFrame with the new columns
# py_df = pd.concat([py_df, py_df_sig], axis=1)
# py_df_sorted = py_df.sort_values(by=['t1w', 't2w', 'fs', 'ksw'])
# 
# n = int(vial_20_m)
# # n = 19
# py_sig = offset_rearranger(py_df_sorted.iloc[n, 4:].values)
# 
# display(py_df_sorted)
# # fig = px.imshow(zspec_quant['ksw'] * full_mask)
# # fig.show()
# # 
# # fig = px.imshow(zspec_quant['t1w'] * full_mask)
# # fig.show()
# # 
# # fig = px.imshow(zspec_quant['t2w'] * full_mask)
# # fig.show()
# # 
# # fig = px.imshow(zspec_quant['match_id'] * full_mask)
# # fig.show()
# 
# py_df_sorted.iloc[n, :]

## 1.2. scan data ##

In [8]:
conc_ls = [['12 mM', '16 mM', '20 mM'],
           ['16 mM', '20 mM', '12 mM'],
           ['12 mM', '16 mM', '20 mM'],
           ['12 mM', '20 mM', '16 mM'],
           ['12 mM', '20 mM', '16 mM'],
           ['20 mM', '12 mM', '16 mM']]

ph_ls = [['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7', 'pH 7']]

tags = [['c', 'b', 'a'], 
        ['b', 'a', 'c'],
        ['b', 'c', 'a'],
        ['c', 'a', 'b'],
        ['c', 'a', 'b'],
        ['a', 'c', 'b']]

tag_x_locs = [[-3,3,-2],
              [-2,1,-2],
              [7, 0, 15],
              [0,-3,0],
              [0,-3,0],
              [0,-3,0]]

tag_y_locs = [[-13,13,14],
              [-13,-13,12],
              [-13,-13,12],
              [-13,12,12],
              [-13,12,12],
              [-13,13,12]]

conc_l = conc_ls[phantom_choice-1]
ph_l = ph_ls[phantom_choice-1]
tag = tags[phantom_choice-1]
tag_x_loc = tag_x_locs[phantom_choice-1]
tag_y_loc = tag_y_locs[phantom_choice-1]

cest_prtcls_names = [['cest_EPI_4uT'],
                     ['cest_EPI_4uT'],
                     ['cest_EPI_4uT'],
                     ['cest_EPI_4uT'],
                     ['cest_EPI_4uT'],
                     ['cest_EPI_4uT']]

temperatures = ['33°C', '37°C', '40°C', '33°C', '37°C', '37°C']
month = ['Dec', 'Dec', 'Dec', 'Feb', 'Feb', 'Dec_og']

B_names = [['4uT'], ['4uT'], ['4uT'], ['4uT'], ['4uT'], ['4uT']]


## 1. z-spectra ##

In [9]:
mean_z_spectra = []
mean_z_spectra_n_c = []
gyro_ratio_hz = 42.5764  # for H [Hz/uT]
b0 = 7

for phantom_i in [0, 1, 2, 3, 4, 5]:  # [0, 1, 2]
    cest_prtcl_names = cest_prtcls_names[phantom_i]
    glu_phantom_fn = glu_phantom_fns[phantom_i]
    B_name = B_names[phantom_i]
    
    # mask
    _, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
    vial_rois, full_mask, _ = mask_roi_finder(bruker_dataset_mask)
    
    # WASSR image
    wassr_dicom_fn, wassr_mrf_files_fn, wassr_bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, 'WASSR')
    wassr_data = dicom_data_arranger(wassr_bruker_dataset, wassr_dicom_fn)
    M0_wassr, arr_wassr_spec = z_spec_rearranger(wassr_data)  # (21, 64, 64)

    wassr_norm = np.divide(arr_wassr_spec, np.where(M0_wassr == 0, 1e-8, M0_wassr))  # (22, 64, 64) full_mask
    offset_hz = offset_rearranger(wassr_bruker_dataset['SatFreqList'].value)
    offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    b0_map = wassr_b0_mapping(wassr_norm, full_mask, w_x=offset_ppm, MainFieldMHz=gyro_ratio_hz*b0)
    
    mean_z_spectrum = np.zeros([3, len(cest_prtcl_names), 57])
    mean_z_spectrum_not_cor = np.zeros([3, len(cest_prtcl_names), 57])
    for cest_prtcl_i, cest_prtcl_name in enumerate(cest_prtcl_names):  # 0.7, 2, 4, 6
        cur_B_name = B_name[cest_prtcl_i]

        # z-spec
        glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, cest_prtcl_name)  # (58/52, 64, 64) [M0,7,-7,6.75,-6.75,...,0.25,-0.25,0]
        cest_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)
        # cest_data_0 = cest_data[0, :, :] * full_mask
        # cest_data_1 = cest_data[1, :, :] * full_mask
        # cest_data_div = cest_data_1 / cest_data_0
        M0_cest, arr_z_spec = z_spec_rearranger(cest_data)
        z_spec_norm = np.divide(arr_z_spec, np.where(M0_cest == 0, 1e-8, M0_cest))  # (51/57, 64, 64) full_mask
        
        # offset vector
        offset_hz = offset_rearranger(bruker_dataset['SatFreqList'].value)
        offset_ppm = offset_hz / (gyro_ratio_hz * b0)
        # offset_ppm = np.round((offset_ppm) / 0.05) * 0.05
        # offset_hz = offset_ppm[1:] * gyro_ratio_hz * b0
    
        b0_cor_zspec = b0_correction(b0_map, z_spec_norm, offset_hz) # have not checked!

        for vial_i, vial_roi in enumerate(vial_rois):
            roi_mask = np.zeros_like(full_mask)
            rr, cc = vial_roi.coords[:, 0], vial_roi.coords[:, 1]
            roi_mask[rr, cc] = 1
            # roi_mask = cv2.cvtColor(roi_mask, cv2.COLOR_BGR2GRAY)
            roi_mask = roi_mask.astype(np.uint8)
            
            for c_i in range(z_spec_norm.shape[0]):
                [[mean_vial]], _ = cv2.meanStdDev(z_spec_norm[c_i, :, :], mask=roi_mask)  # before b0 correction
                mean_z_spectrum_not_cor[vial_i,cest_prtcl_i,c_i] = mean_vial
                [[mean_vial]], _ = cv2.meanStdDev(b0_cor_zspec[c_i, :, :], mask=roi_mask)
                mean_z_spectrum[vial_i,cest_prtcl_i,c_i] = mean_vial

    mean_z_spectra.append(mean_z_spectrum)
    mean_z_spectra_n_c.append(mean_z_spectrum_not_cor)


WASSR B0 mapping took 3.440 seconds
WASSR B0 mapping took 3.507 seconds
WASSR B0 mapping took 3.473 seconds
WASSR B0 mapping took 3.378 seconds
WASSR B0 mapping took 3.358 seconds
WASSR B0 mapping took 3.337 seconds


In [12]:
for phantom_i in [0, 1, 2, 3, 5]:
    cest_prtcl_names = cest_prtcls_names[phantom_i]  # 3.6, 4, 6
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    conc_l = conc_ls[phantom_i]
    ph_l = ph_ls[phantom_i]
    tag = tags[phantom_i]
    abc_id = [tag.index('a'), tag.index('b'), tag.index('c')]

    fig = make_subplots(rows=1, cols=len(cest_prtcl_names),
                        subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.01)
    fig_2 = make_subplots(rows=1, cols=len(cest_prtcl_names),
                          subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.05)

    mean_z_spectrum = mean_z_spectra[phantom_i]
    mean_z_spectrum_not_cor = mean_z_spectra_n_c[phantom_i]
    # Iterate over each channel
    for B_idx in range(len(cest_prtcl_names)):
        # Iterate over each image
        for tag_i, cur_tag in enumerate(['a', 'b', 'c']):
            vial_i = tag.index(cur_tag)
            z_vial_B = mean_z_spectrum[vial_i,B_idx,:]
            z_vial_B_not_cor = mean_z_spectrum_not_cor[vial_i,B_idx,:]
            
            ppm_lim = 7

            # MTR asym calculation
            cha_n = len(z_vial_B)
            mid_i = int(cha_n/2 - 1)
            last_i = int(cha_n-1)
            positives = z_vial_B[0:(mid_i+2)]  # positives to 0 (including)
            negatives = z_vial_B[last_i:mid_i:-1]  # negatives to 0 (including)

            MTR_asym = negatives-positives
            ppm_asym = np.linspace(ppm_lim, 0, num=(mid_i+2))     

            ppm = np.linspace(ppm_lim, -ppm_lim, num=cha_n)

            col_idx = B_idx + 1
            fig.add_trace(go.Scatter(x=ppm, y=z_vial_B, line=dict(color=colors[tag_i]), name=f'{cur_tag} {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
            # fig.add_trace(go.Scatter(x=ppm, y=z_vial_B_not_cor, line=dict(color='gray'), name=f'{cur_tag} {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)  # later delete!
            
            fig.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_i]), name=f'{conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
            fig.update_xaxes(title_text='ppm', row=1, col=col_idx)
            
            fig_2.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_i]), name=f'{conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
            fig_2.update_xaxes(title_text='ppm', row=1, col=col_idx)
            
    fig.update_yaxes(title_text='$M_{sat}/M_0$', row=1, col=1, title_standoff=2)
    fig_2.update_yaxes(title_text='MTR-asym', row=1, col=1)
    
    # Update layout
    fig.update_layout(template='plotly_white',  # Set the theme to plotly white
                      title_text=f'Phantom {month[phantom_i]} - {temperatures[phantom_i]} Z-spectrum',
                      height=250, width=500,
                      title=dict(x=0.02, y=0.97),
                      margin=dict(l=45, r=0, t=50, b=0)
                      )  # Adjust the title position
    
    fig_2.update_layout(template='plotly_white',  # Set the theme to plotly white
                  title_text=f'Phantom {month[phantom_i]} - {temperatures[phantom_i]} MTR-asym',
                  height=250, width=350,
                  title=dict(x=0.02, y=0.97),
                  margin=dict(l=0, r=0, t=60, b=0)
                        )  # Adjust the title position

    # Set axes for z-spectrum
    fig.update_xaxes(autorange='reversed',tickmode='linear', tick0=0, dtick=1)
    fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.2, range=[-0.1, 1])
    
    # # Set axes for MTR-asym
    fig_2.update_xaxes(autorange='reversed', dtick=1)
    # fig_2.update_xaxes(tickvals=list(range(5)), ticktext=list(map(str, range(4, -1, -1))))
    fig_2.update_yaxes(tickmode='linear', tick0=0, dtick=0.05, range=[0, 0.25], tickformat='.0%')    

    # Show only specific legend groups
    for trace in fig.data:
        trace.showlegend = (trace.legendgroup == '1_1_1')
        
    # Show only specific legend groups
    for trace in fig_2.data:
        trace.showlegend = (trace.legendgroup == '1_1_2')

    fig.show()
    fig_2.show()
    
    pio.write_image(fig, f'images/z_spec/zspec_phantom_{phantom_i+1}.jpeg')
    pio.write_image(fig_2, f'images/z_spec/mtr_phantom_{phantom_i+1}.jpeg')
